In [1]:
using Pkg
Pkg.activate("/media/mat/HDD/AdaptiveTransportMap/")

 Activating environment at `/media/mat/HDD/AdaptiveTransportMap/Project.toml`


In [2]:
using Revise
using AdaptiveTransportMap
using LinearAlgebra
import LinearAlgebra: ldiv!, dot
using Test
using ForwardDiff
using SpecialFunctions
using BenchmarkTools
using TransportMap
using QuadGK
using AdaptiveTransportMap: derivative, vander, transform!, evaluate, ncoeff, optimize
using DiffResults
using Distributions
using Random
using LoopVectorization
using Optim
using NLsolve
using Polynomials
using Optim: InverseDiagonal
using MLDataUtils
using Test

┌ Info: Precompiling AdaptiveTransportMap [bdf749b0-1400-4207-80d3-e689c0e3f03d]
└ @ Base loading.jl:1260


### Expanded function

In [3]:
Nx = 3
Ne = 8
X = randn(Nx, Ne)'

8×3 Adjoint{Float64,Array{Float64,2}}:
  0.36704    1.40948   -0.075985
  1.23996   -0.469302   0.00113042
 -0.830775   0.644367  -1.13684
 -1.60413   -1.19758    0.80183
 -0.726377  -0.903612   0.413753
  0.291923  -0.735504  -1.09496
  0.872641  -1.20111    0.831016
  1.17806   -1.96244    0.729214

In [35]:
Nx = 3
Ne = 8

X = Matrix([-0.899679   0.149212  -0.203239;
     -1.20342   -0.196523  -0.484925;
     -1.03261   -1.04501    0.367875;
     -0.343662   0.424117   0.720608;
      0.32033    0.345181   0.739602;
      1.0377    -1.029     -0.610308;
      0.36607    2.42988   -0.0927185;
     -1.15715    0.803241   1.71867]')

# X = randn(Nx, Ne)

B = MultiBasis(CstProHermite(3; scaled =true), Nx)

idx = [0 0 0; 0 0 1; 0 1 0; 0 1 1; 0 1 2; 1 0 0]
Nψ = size(idx,1)
coeff =  randn(Nψ)

f = ExpandedFunction(B, idx, coeff)

out = zeros(Ne, Nψ);

In [36]:
@btime evaluate_basis!($out, $f, $X)

  636.602 ns (27 allocations: 2.13 KiB)


8×6 Array{Float64,2}:
 1.0  0.62513   0.628113  0.392652   -0.0798022  0.515909
 1.0  0.595558  0.62555   0.372551   -0.180659   0.439761
 1.0  0.610607  0.480715  0.293528    0.107982   0.483821
 1.0  0.554722  0.603845  0.334966    0.241379   0.613242
 1.0  0.550889  0.613082  0.33774     0.249793   0.615622
 1.0  0.575458  0.484722  0.278937   -0.170238   0.482548
 1.0  0.630263  0.144345  0.0909753  -0.0084351  0.610809
 1.0  0.30182   0.537531  0.162237    0.278833   0.451934

In [6]:
evaluate_basis!(out, f, X)

8×6 Array{Float64,2}:
 1.0  0.62513   0.628113  0.392652   -0.0798022  0.515909
 1.0  0.595558  0.62555   0.372551   -0.180659   0.439761
 1.0  0.610607  0.480715  0.293528    0.107982   0.483821
 1.0  0.554722  0.603845  0.334966    0.241379   0.613242
 1.0  0.550889  0.613082  0.33774     0.249793   0.615622
 1.0  0.575458  0.484722  0.278937   -0.170238   0.482548
 1.0  0.630263  0.144345  0.0909753  -0.0084351  0.610809
 1.0  0.30182   0.537531  0.162237    0.278833   0.451934

In [261]:
@btime evaluate_basis!($out, $f, $X)

  357.426 ns (13 allocations: 1.14 KiB)


8×6 Array{Float64,2}:
 1.0  0.62513   0.628113  0.392652   -0.0798022  0.515909
 1.0  0.595558  0.62555   0.372551   -0.180659   0.439761
 1.0  0.610607  0.480715  0.293528    0.107982   0.483821
 1.0  0.554722  0.603845  0.334966    0.241379   0.613242
 1.0  0.550889  0.613082  0.33774     0.249793   0.615622
 1.0  0.575458  0.484722  0.278937   -0.170238   0.482548
 1.0  0.630263  0.144345  0.0909753  -0.0084351  0.610809
 1.0  0.30182   0.537531  0.162237    0.278833   0.451934

In [255]:
@btime evaluate_basis!($out, $f, $X)

  443.899 ns (12 allocations: 1.14 KiB)


8×6 Array{Float64,2}:
 1.0  0.62513   0.628113  0.392652   -0.0798022  0.515909
 1.0  0.595558  0.62555   0.372551   -0.180659   0.439761
 1.0  0.610607  0.480715  0.293528    0.107982   0.483821
 1.0  0.554722  0.603845  0.334966    0.241379   0.613242
 1.0  0.550889  0.613082  0.33774     0.249793   0.615622
 1.0  0.575458  0.484722  0.278937   -0.170238   0.482548
 1.0  0.630263  0.144345  0.0909753  -0.0084351  0.610809
 1.0  0.30182   0.537531  0.162237    0.278833   0.451934

In [250]:
@btime evaluate_basis!($out, $f, $X)

  428.472 ns (12 allocations: 1.14 KiB)


8×6 Array{Float64,2}:
 1.0  0.62513   0.628113  0.392652   -0.0798022  0.515909
 1.0  0.595558  0.62555   0.372551   -0.180659   0.439761
 1.0  0.610607  0.480715  0.293528    0.107982   0.483821
 1.0  0.554722  0.603845  0.334966    0.241379   0.613242
 1.0  0.550889  0.613082  0.33774     0.249793   0.615622
 1.0  0.575458  0.484722  0.278937   -0.170238   0.482548
 1.0  0.630263  0.144345  0.0909753  -0.0084351  0.610809
 1.0  0.30182   0.537531  0.162237    0.278833   0.451934

In [239]:
@btime evaluate_basis!($out, $f, $X)

  718.075 ns (30 allocations: 2.27 KiB)


8×6 Array{Float64,2}:
 1.0  0.62513   0.628113  0.392652   -0.0798022  0.515909
 1.0  0.595558  0.62555   0.372551   -0.180659   0.439761
 1.0  0.610607  0.480715  0.293528    0.107982   0.483821
 1.0  0.554722  0.603845  0.334966    0.241379   0.613242
 1.0  0.550889  0.613082  0.33774     0.249793   0.615622
 1.0  0.575458  0.484722  0.278937   -0.170238   0.482548
 1.0  0.630263  0.144345  0.0909753  -0.0084351  0.610809
 1.0  0.30182   0.537531  0.162237    0.278833   0.451934

### HermiteMap

In [60]:
Nx = 20
Ne = 300
m = 10
X = randn(Nx, Ne) .* randn(Nx, Ne) .+ rand(Nx);
X0 = deepcopy(X);

In [61]:
M = HermiteMap(m, X);

In [63]:
out = zero(X);
@time optimize(M, X, 6; apply_rescaling = true);

  0.326792 seconds (593.41 k allocations: 732.832 MiB, 11.31% gc time)


In [12]:
@code_warntype  randn(Nx) 
#         M = HermiteMap(m, X);    
#         optimize(M, X, 6; apply_rescaling = true)

Variables
  #self#::Core.Compiler.Const(randn, false)
  dims::Tuple{Int64}

Body::Array{Float64,1}
1 ─ %1 = Random.default_rng()::MersenneTwister
│   %2 = Core.tuple(%1, Random.Float64)::Core.Compiler.PartialStruct(Tuple{MersenneTwister,DataType}, Any[MersenneTwister, Core.Compiler.Const(Float64, false)])
│   %3 = Core._apply_iterate(Base.iterate, Random.randn, %2, dims)::Array{Float64,1}
└──      return %3


In [58]:
function timing()
    @btime begin
        Nx = 10
        Ne = 300
        m = 10
        X = randn(Nx, Ne) .* randn(Nx, Ne) .+ rand(Nx);
        X0 = deepcopy(X);   
#         M = HermiteMap(m, X);    
#         optimize(M, X, 4; apply_rescaling = true)
    end
    
    @btime begin
        Nx = 20
        Ne = 300
        m = 10
        X = randn(Nx, Ne) .* randn(Nx, Ne) .+ rand(Nx);
        X0 = deepcopy(X);   
        M = HermiteMap(m, X);    
        optimize(M, X, 10; apply_rescaling = true)
    end
end

timing (generic function with 1 method)

In [59]:
timing();

  31.415 μs (11 allocations: 94.58 KiB)
  893.217 ms (1209838 allocations: 1.87 GiB)


### Greedyfit

In [32]:
Nx = 10
Ne = 300
m = 20

# X = Matrix([0.267333   1.43021;
#       0.364979   0.607224;
#      -1.23693    0.249277;
#      -2.0526     0.915629;
#      -0.182465   0.415874;
#       0.412907   1.01672;
#       1.41332   -0.918205;
#       0.766647  -1.00445]');

X = randn(Nx, Ne) .* randn(Nx, Ne) .+ rand(Nx)
L = LinearTransform(X; diag = true)

transform!(L, X)

10×300 Array{Float64,2}:
  0.297382    -0.117953  -1.10087      0.106203   …   0.641164   -0.0177259
 -0.0678983   -0.744166  -0.0852826   -1.24427        0.290751   -0.192203
  0.178115     2.94028    0.188398     1.46241        0.990471   -3.60733
 -1.54916      0.117499  -0.0928214   -0.222249      -0.14336    -0.487068
 -0.00268947   1.04019   -0.299954    -2.33092        0.0344107  -0.000892016
  1.06619     -0.824913  -0.986831    -0.412307   …   0.401896   -0.526111
 -0.256012     0.954342   0.00187053  -0.369885       0.675455    1.73183
 -0.129779     0.167482   1.23641      0.0573571     -0.370129   -0.701837
 -1.17288      1.70785   -0.0999442   -0.0332631     -1.336       2.40839
 -0.457943    -0.515946  -0.320347    -0.644992       1.33783    -0.291876

In [33]:
@btime greedyfit($m, $Nx, $X, $8; verbose = false);

  17.678 ms (31741 allocations: 42.76 MiB)


In [30]:
@time greedyfit(m, Nx, X, 8; verbose = false);

  0.045642 seconds (41.58 k allocations: 64.954 MiB, 16.33% gc time)


In [34]:
@time greedyfit(m, Nx, X, 8; verbose = false);

  0.044889 seconds (41.58 k allocations: 64.954 MiB, 12.20% gc time)


In [35]:
@btime greedyfit($m, $Nx, $X, $8; verbose = false);

  29.638 ms (41583 allocations: 64.95 MiB)
